In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 47.9 MB/s 
     |████████████████████████████████| 895 kB 45.2 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 6.8 MB 38.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [ ]:
import sentencepiece as sp
sp.SentencePieceTrainer.train(input='/content/drive/MyDrive/Shadhu _Kobita/full_corpas.txt',model_prefix='spiece', vocab_size=101975)

In [ ]:
import os
os.mkdir('Own_Model')
os.rename('spiece.model','Own_Model/spiece.model')
os.rename('spiece.vocab','Own_Model/spiece.vocab')

In [ ]:
from transformers import AlbertForMaskedLM,AlbertConfig,AlbertTokenizer,AutoTokenizer, AutoModel

In [ ]:
Bangla_tokenizer = AlbertTokenizer.from_pretrained('Own_Model')

file Own_Model/config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
Bangla_tokenizer.save_pretrained('Own_Model')

('Own_Model/tokenizer_config.json',
 'Own_Model/special_tokens_map.json',
 'Own_Model/spiece.model',
 'Own_Model/added_tokens.json')

In [ ]:
from transformers import LineByLineTextDataset
dataset = LineByLineTextDataset(
    tokenizer=Bangla_tokenizer,
    file_path="/content/drive/MyDrive/Shadhu _Kobita/full_corpas.txt",
    block_size=256,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=Bangla_tokenizer,mlm=True, mlm_probability=0.15)

In [ ]:
config=AlbertConfig(
    vocab_size=101975,
    embedding_size=128,
    hidden_size=128,
    num_hidden_layers=2,
    num_attention_heads=2,
    intermediate_size=512,
    hidden_act="gelu",
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    max_position_embeddings=512,
    type_vocab_size=2,
    initializer_range=0.02,
    layer_norm_eps=1e-12,
    pad_token_id=0,
    bos_token_id=2,
    eos_token_id=3
)

In [ ]:
config.save_pretrained('Own_Model')

In [ ]:
config=AlbertConfig.from_pretrained('Own_Model')
own_model = AlbertForMaskedLM(config=config)

In [ ]:
from transformers import Trainer, TrainingArguments
batch_size=4
training_args = TrainingArguments(
    output_dir='bangla_albert',
    overwrite_output_dir=True,
    num_train_epochs=1,
    learning_rate=5e-05,
    per_device_train_batch_size=batch_size,
    save_steps=len(dataset)/batch_size,
    save_total_limit=2,
    prediction_loss_only=True
)

trainer = Trainer(
    model=own_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 233695
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 58424


Step,Training Loss
500,10.202800
1000,8.461800
1500,8.135900
2000,8.061900
2500,7.993000
3000,7.948700
3500,7.843600
4000,7.887500
4500,7.867100
5000,7.756200




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=58424, training_loss=7.361065008567487, metrics={'train_runtime': 2816.8294, 'train_samples_per_second': 82.964, 'train_steps_per_second': 20.741, 'total_flos': 32334490168344.0, 'train_loss': 7.361065008567487, 'epoch': 1.0})